In [1]:
import cv2

In [2]:
net = cv2.dnn.readNetFromDarknet("demo/yolov4-tiny.cfg", "demo/yolov4-tiny.weights")

In [3]:
layer_names = net.getLayerNames()
layer_names

('conv_0',
 'bn_0',
 'leaky_1',
 'conv_1',
 'bn_1',
 'leaky_2',
 'conv_2',
 'bn_2',
 'leaky_3',
 'slice_3',
 'conv_4',
 'bn_4',
 'leaky_5',
 'conv_5',
 'bn_5',
 'leaky_6',
 'concat_6',
 'conv_7',
 'bn_7',
 'leaky_8',
 'concat_8',
 'pool_9',
 'conv_10',
 'bn_10',
 'leaky_11',
 'slice_11',
 'conv_12',
 'bn_12',
 'leaky_13',
 'conv_13',
 'bn_13',
 'leaky_14',
 'concat_14',
 'conv_15',
 'bn_15',
 'leaky_16',
 'concat_16',
 'pool_17',
 'conv_18',
 'bn_18',
 'leaky_19',
 'slice_19',
 'conv_20',
 'bn_20',
 'leaky_21',
 'conv_21',
 'bn_21',
 'leaky_22',
 'concat_22',
 'conv_23',
 'bn_23',
 'leaky_24',
 'concat_24',
 'pool_25',
 'conv_26',
 'bn_26',
 'leaky_27',
 'conv_27',
 'bn_27',
 'leaky_28',
 'conv_28',
 'bn_28',
 'leaky_29',
 'conv_29',
 'permute_30',
 'yolo_30',
 'identity_31',
 'conv_32',
 'bn_32',
 'leaky_33',
 'upsample_33',
 'concat_34',
 'conv_35',
 'bn_35',
 'leaky_36',
 'conv_36',
 'permute_37',
 'yolo_37')

In [4]:
out_layers_indexes = net.getUnconnectedOutLayers()
out_layers_indexes

array([66, 78], dtype=int32)

In [5]:
out_layers = [layer_names[index - 1] for index in out_layers_indexes]
out_layers

['yolo_30', 'yolo_37']

In [6]:
with open("demo/coco.names.txt") as file:
    classes = file.read().split("\n")
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush',
 '']

In [7]:
classes_to_look_for = ["truck", "person"]

In [14]:
import numpy as np


def draw_object_bounding_box(image_to_process, index, box):
    """
    Рисование границ объекта с подписями
    :param image_to_process: исходное изображение
    :param index: индекс определённого с помощью YOLO класса объекта
    :param box: координаты области вокруг объекта
    :return: изображение с отмеченными объектами
    """
    x, y, w, h = box
    start = (x, y)
    end = (x + w, y + h)
    color = (0, 255, 0)
    width = 2
    final_image = cv2.rectangle(image_to_process, start, end, color, width)

    start = (x, y - 10)
    font_size = 1
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 2
    text = classes[index]
    final_image = cv2.putText(final_image, text, start, font, font_size, color, width, cv2.LINE_AA)

    return final_image


def draw_object_count(image_to_process, objects_count):
    """
    Подпись количества найденных объектов на изображении
    :param image_to_process: исходное изображение
    :param objects_count: количество объектов искомого класса
    :return: изображение с подписаным количеством найденных объектов
    """

    start = (45, 150)
    font_size = 1.5
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 3
    text = "Objects found: " + str(objects_count)

    # вывод текста с обводкой (чтобы было видно при разном освещении картинки)
    white_color = (255, 255, 255)
    black_outline_color = (0, 0, 0)
    final_image = cv2.putText(image_to_process, text, start, font, font_size, black_outline_color, width * 3, cv2.LINE_AA)
    final_image = cv2.putText(final_image, text, start, font, font_size, white_color, width, cv2.LINE_AA)

    return final_image


def apply_yolo_object_detection(image_to_process):
    """
    Распознавание и определение координат объектов на изображении
    :param image_to_process: исходное изображение
    :return: изображение с размеченными объектами и подписями к ним
    """
    height, width, depth = image_to_process.shape
    blob = cv2.dnn.blobFromImage(image_to_process, 1 / 255, (608, 608), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)

    print(outs)
    print(len(outs))
    for x in outs:
        print(len(x))

    class_indexes, class_scores, boxes = ([] for i in range(3))
    objects_count = 0

    # запуск поиска объектов на изображении
    for out in outs:
        for obj in out:
            scores = obj[5:]
            class_index = np.argmax(scores)
            class_score = scores[class_index]
            if class_score > 0:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                obj_width = int(obj[2] * width)
                obj_height = int(obj[3] * height)

                box = [center_x - obj_width // 2, center_y - obj_height // 2, obj_width, obj_height]
                boxes.append(box)
                class_indexes.append(class_index)
                class_scores.append(float(class_score))

    # проведение выборки
    chosen_boxes = cv2.dnn.NMSBoxes(boxes, class_scores, 0.0, 0.4)

    print(chosen_boxes)
    print(class_indexes)

    for box_index in chosen_boxes:
        box_index = box_index
        box = boxes[box_index]
        class_index = class_indexes[box_index]

        # debug-рисование объектов, входящих в искомые классы
        if classes[class_index] in classes_to_look_for:
            objects_count += 1
            image_to_process = draw_object_bounding_box(image_to_process, class_index, box)

    final_image = draw_object_count(image_to_process, objects_count)

    return final_image


In [15]:
    try:
        # применение методов распознавания объектов на изображении от YOLO
        image = cv2.imread("demo/assets/truck_captcha.png")
        image = apply_yolo_object_detection(image)

        # # вывод обработанного изображения на экран
        # cv2.imshow("Image", image)
        # if cv2.waitKey(0):
        #     cv2.destroyAllWindows()

    except KeyboardInterrupt:
        pass

(array([[0.04078696, 0.03125866, 0.11186892, ..., 0.        , 0.        ,
        0.        ],
       [0.03703755, 0.02410808, 0.12978752, ..., 0.        , 0.        ,
        0.        ],
       [0.02969689, 0.02501762, 0.47472197, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.961629  , 0.9700703 , 0.13243048, ..., 0.        , 0.        ,
        0.        ],
       [0.96200985, 0.95834273, 0.1637088 , ..., 0.        , 0.        ,
        0.        ],
       [0.9695973 , 0.95818186, 0.36141846, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.01262581, 0.01154043, 0.04110672, ..., 0.        , 0.        ,
        0.        ],
       [0.01351764, 0.02039025, 0.05433187, ..., 0.        , 0.        ,
        0.        ],
       [0.01111054, 0.01947958, 0.1707725 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9861131 , 0.9834872 , 0.03457957, ..., 0.        , 0.        ,
        0.        ],
       [0.9842321 